In [1]:
import os
import requests
import re
import math
import easier as ezr
import itertools
from pprint import pprint
%config Completer.use_jedi = False

In [5]:
class GithubBase:
    BASE_URL = 'https://api.github.com/gists'
    USER_URL = 'https://api.github.com/users/robdmc/gists'
    REX_NEXT = re.compile(r'.*<(https.*?)>; rel="next"')
    PER_PAGE = 99
#     PER_PAGE = 9
    MAX_GISTS = 10_000

    def __init__(self):
        self.access_token = os.environ['GIST_TOKEN']

    def blob_generator(self):
        for page in range(int(math.ceil(self.MAX_GISTS / self.PER_PAGE))):
            params = {
                'per_page': self.PER_PAGE,
                'page': page,
            }
            headers = {
                'Authorization': f'token {self.access_token}',
                'accept': 'application/vnd.github.v3+json'
            }
            resp = requests.get(self.USER_URL, params=params, headers=headers)
            self.latest_resp = resp
            blobs = resp.json()
            if blobs:
                yield blobs
            else:
                break
                
    def get_list(self):
        rec_list = []
        for ll in self.blob_generator():
            rec_list.extend(ll)
        return rec_list
    
    @ezr.pickle_cached_container()
    def ll(self):
        return self.get_list()
    
    def get_parsed_blob_list(self):
        keys = [
            'url',
            'description',
            'created_at',
            'updated_at',

        ]
        records = []
        
        # for item in self.blob_generator():
        for item in self.ll[:3]:
            record_template = {k: item[k] for k in keys}
            for file in item['files'].values():
                record = record_template.copy()
                record['file'] = file['filename']
                record['language'] = file.get('language', 'unkown')
                record['file_url'] = file['raw_url']
                record['size'] = file.get('size', 0)
                records.append(record)
                
        return records



gh = GithubBase()
recs = gh.get_parsed_blob_list()
print(type(recs))
print(len(recs))

<class 'list'>
3


In [4]:
# recs

In [11]:
type(resp.headers)

requests.structures.CaseInsensitiveDict

In [37]:
blob = resp.json()

keys = [
    'url',
    'description',
    'created_at',
    'updated_at',
    
]
records = []
for item in blob:
    record_template = {k: item[k] for k in keys}
    for file in item['files'].values():
        record = record_template.copy()
#         print(file)
        record['file'] = file['filename']
        record['language'] = file.get('language', 'unkown')
        record['file_url'] = file['raw_url']
        record['size'] = file.get('size', 0)
        records.append(record)
#     pprint(item)

rec = records[0]
rec


{'url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84',
 'description': 'Visidata Monstrosity',
 'created_at': '2021-01-17T17:14:45Z',
 'updated_at': '2021-01-17T19:55:10Z',
 'file': 'monstrous_visidatarc.py',
 'language': 'Python',
 'file_url': 'https://gist.githubusercontent.com/robdmc/611b57470ca1522c4f3c496a7c967a84/raw/cb7e9a75c640e1c9b0c3bb39a2f6beda40b6f327/monstrous_visidatarc.py',
 'size': 1999}

In [16]:
blob[0]

{'url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84',
 'forks_url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84/forks',
 'commits_url': 'https://api.github.com/gists/611b57470ca1522c4f3c496a7c967a84/commits',
 'id': '611b57470ca1522c4f3c496a7c967a84',
 'node_id': 'MDQ6R2lzdDYxMWI1NzQ3MGNhMTUyMmM0ZjNjNDk2YTdjOTY3YTg0',
 'git_pull_url': 'https://gist.github.com/611b57470ca1522c4f3c496a7c967a84.git',
 'git_push_url': 'https://gist.github.com/611b57470ca1522c4f3c496a7c967a84.git',
 'html_url': 'https://gist.github.com/611b57470ca1522c4f3c496a7c967a84',
 'files': {'monstrous_visidatarc.py': {'filename': 'monstrous_visidatarc.py',
   'type': 'application/x-python',
   'language': 'Python',
   'raw_url': 'https://gist.githubusercontent.com/robdmc/611b57470ca1522c4f3c496a7c967a84/raw/cb7e9a75c640e1c9b0c3bb39a2f6beda40b6f327/monstrous_visidatarc.py',
   'size': 1999}},
 'public': True,
 'created_at': '2021-01-17T17:14:45Z',
 'updated_at': '2021-01-17T19:55:

In [2]:
from visidata import SequenceSheet, BaseSheet, Path, Column, asyncthread, TableSheet
from visidata import vd


In [3]:

# def rowtype(*args, **kwargs):
class RobSheet(TableSheet):
    _rowtype = object  # rowdef: list of values

    

    def iterload(self):
        for nn in range(3):
            yield  [nn, nn, nn]

    #     @asyncthread
    def reload(self):
        self.columns = []
        for col in ['a', 'b', 'c']:
            self.addColumn(Column(col))
        
        self.rows = []
        for row in self.iterload():
            self.addRow(row)
            
        

In [4]:
rs = RobSheet()
t = rs.reload()

# for col in rs.it:
#     print(repr(col))


In [5]:
for col in rs.columns:
    print(col.name)

a
b
c


In [6]:
for typedvals in rs.iterdispvals(format=False):

    for col, val in typedvals.items():
        print(col.name, val, type(val))

a [0, 0, 0] <class 'list'>
b [0, 0, 0] <class 'list'>
c [0, 0, 0] <class 'list'>
a [1, 1, 1] <class 'list'>
b [1, 1, 1] <class 'list'>
c [1, 1, 1] <class 'list'>
a [2, 2, 2] <class 'list'>
b [2, 2, 2] <class 'list'>
c [2, 2, 2] <class 'list'>


In [154]:
vd.save_csv(Path('/tmp/jerry.csv'), rs)

In [161]:
nr = rs.newRow()

In [162]:
nr[['a', 'b']]

TypeError: unhashable type: 'list'

In [160]:
nr

BasicRow(<function visidata.sheets.BasicRow.__init__.<locals>.<lambda>()>,
         {'rob': None})